In [ ]:
# === Setup ===
import sys, pathlib, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import topotoolbox as tt3
import topotoolbox._stream as _stream



# Add project root so "channel_heads" package is visible
project_root = pathlib.Path("/Users/guypi/Projects/channel-heads")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Imports from your channel_heads package
from channel_heads.first_meet_pairs_for_outlet import first_meet_pairs_for_outlet
from channel_heads.coupling_analysis import CouplingAnalyzer
from channel_heads.stream_utils import outlet_node_ids_from_streampoi
from channel_heads.plotting_utils import (
    plot_coupled_pair,
    plot_outlet_view,
    plot_all_coupled_pairs_for_outlet,
    plot_all_coupled_pairs_for_outlet_3d
)

In [2]:
def compute_coupling_all_outlets(s, fd, dem, outlets=None, warmup=True, connectivity=8):
    """
    Build a single DataFrame of coupling results for all (or selected) outlets.

    Columns:
      outlet, confluence, head_1, head_2, touching, overlap_px, contact_px, size1_px, size2_px
    """
    # 0) pick outlets
    if outlets is None:
        outlets = outlet_node_ids_from_streampoi(s)
    outlets = [int(o) for o in outlets]

    # 1) analyzer (we’ll reuse it; clear its cache per outlet to bound memory)
    an = CouplingAnalyzer(fd, s, dem, connectivity=connectivity)

    dfs = []
    for idx, o in enumerate(outlets, 1):
        # print small progress breadcrumb
        print(f"[{idx}/{len(outlets)}] outlet={o}", flush=True)

        # 2) first-meet pairs for this outlet
        pairs_at_confluence, basin_heads = first_meet_pairs_for_outlet(s, o)

        # 3) optional warm-up: precompute masks for heads in this basin
        if warmup and basin_heads:
            an._mask_cache.clear()          # keep cache limited per outlet
            # touch each head once to populate cache
            for h in basin_heads:
                _ = an.influence_mask(int(h))

        # 4) evaluate pairs → per-outlet DataFrame
        df_o = an.evaluate_pairs_for_outlet(o, pairs_at_confluence)
        if not df_o.empty:
            dfs.append(df_o)

    # 5) concat
    if not dfs:
        cols = ["outlet","confluence","head_1","head_2","touching","overlap_px","contact_px","size1_px","size2_px"]
        return pd.DataFrame(columns=cols)

    df_all = pd.concat(dfs, ignore_index=True)
    # nice ordering
    df_all.sort_values(["outlet","confluence","head_1","head_2"], inplace=True, ignore_index=True)
    return df_all

In [ ]:
from pathlib import Path


DATA_DIR = Path("/Users/guypi/Projects/channel-heads/data/cropped_DEMs")

# Get all .tif files from data directory
tif_files = sorted(DATA_DIR.glob("*.tif"))

print(f"Found {len(tif_files)} .tif files in {DATA_DIR}:")
for tif_file in tif_files:
    print(f" - {tif_file.name}")

for tif_file in tif_files:
    print(f"\nProcessing Basin: {tif_file.name.split("_")[0]}")
    dem = tt3.read_tif(tif_file)
    fd = tt3.FlowObject(dem)
    s = tt3.StreamObject(fd, threshold=1000)
    an = CouplingAnalyzer(fd, s, dem)
    outs = outlet_node_ids_from_streampoi(s)
    df_touching = compute_coupling_all_outlets(s, fd, dem, outlets=outs, warmup=True, connectivity=8)
    
    

In [4]:
from pathlib import Path

DATA_DIR = Path("/Users/guypi/Projects/channel-heads/data/cropped_DEMs")
OUTPUT_DIR = Path("/Users/guypi/Projects/channel-heads/data/outputs")

# Get all .tif files from data directory
tif_files = sorted(DATA_DIR.glob("*.tif"))

print(f"Found {len(tif_files)} .tif files in {DATA_DIR}:")
for tif_file in tif_files:
    print(f" - {tif_file.name}")

for tif_file in tif_files:
    # שם הרכס - אפשר לכוונן לפי קונבנציה של שמות הקבצים
    basin_name = tif_file.stem.split("_")[0]
    print(f"\nProcessing Basin: {basin_name}")

    # קריאת DEM ובניית אובייקטי זרימה
    dem = tt3.read_tif(tif_file)
    fd = tt3.FlowObject(dem)
    s = tt3.StreamObject(fd, threshold=1000)

    # אנליזה
    an = CouplingAnalyzer(fd, s, dem)
    outs = outlet_node_ids_from_streampoi(s)

    df_touching = compute_coupling_all_outlets(
        s,
        fd,
        dem,
        outlets=outs,
        warmup=True,
        connectivity=8,
    )

    # הגדרת תיקיית output לרכס הזה
    basin_dir = OUTPUT_DIR / basin_name
    basin_dir.mkdir(parents=True, exist_ok=True)

    # שם הקובץ ל־CSV - אפשר לשנות אם תרצה שמות שונים
    out_csv = basin_dir / "coupling_touching.csv"

    # אם יש קובץ קודם - מוחקים
    if out_csv.exists():
        out_csv.unlink()

    # כתיבת התוצאה כ־CSV חדש
    df_touching.to_csv(out_csv, index=False)

    print(f"Saved coupling results to: {out_csv}")

Found 6 .tif files in /Users/guypi/Projects/channel-heads/data/cropped_DEMs:
 - CalnAlpine_strm_crop.tif
 - Daqing_strm_crop.tif
 - Humboldt_strm_crop.tif
 - Inyo_strm_crop.tif
 - Kammanasie_strm_crop.tif
 - Luliang_strm_crop.tif

Processing Basin: CalnAlpine
[1/19] outlet=0
[2/19] outlet=1
[3/19] outlet=2
[4/19] outlet=196
[5/19] outlet=221
[6/19] outlet=592
[7/19] outlet=934
[8/19] outlet=1251
[9/19] outlet=1305
[10/19] outlet=1654
[11/19] outlet=1673
[12/19] outlet=1718
[13/19] outlet=1752
[14/19] outlet=2151
[15/19] outlet=2152
[16/19] outlet=2153
[17/19] outlet=2154
[18/19] outlet=2155
[19/19] outlet=2156
Saved coupling results to: /Users/guypi/Projects/channel-heads/data/outputs/CalnAlpine/coupling_touching.csv

Processing Basin: Daqing
[1/16] outlet=0
[2/16] outlet=10
[3/16] outlet=50
[4/16] outlet=146
[5/16] outlet=152
[6/16] outlet=239
[7/16] outlet=404
[8/16] outlet=418
[9/16] outlet=446
[10/16] outlet=451
[11/16] outlet=478
[12/16] outlet=570
[13/16] outlet=660
[14/16] outle

In [8]:
df = pd.read_csv("/Users/guypi/Projects/channel-heads/data/outputs/CalnAlpine/coupling_touching.csv")
df.head()

,outlet,confluence,head_1,head_2,touching,overlap_px,contact_px,size1_px,size2_px
0,0,148,707,1132,True,0,24,1063,1063
1,1,43,464,551,False,0,0,1011,1036
2,1,43,464,1159,False,0,0,1011,1002
3,1,43,551,745,False,0,0,1036,1051
4,1,43,551,1202,False,0,0,1036,1005
